In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.io as pio

from sklearn.cluster import AgglomerativeClustering
from scipy.cluster import hierarchy
from sklearn.metrics import silhouette_score
from base64 import b64encode

from sklearn.cluster import DBSCAN

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples


In [ ]:
d1 = pd.read_csv("https://raw.githubusercontent.com/cmparlettpelleriti/CPSC392ParlettPelleriti/master/Data/KMEM1.csv")
d2 = pd.read_csv("https://raw.githubusercontent.com/cmparlettpelleriti/CPSC392ParlettPelleriti/master/Data/KMEM2.csv")
d3 = pd.read_csv("https://raw.githubusercontent.com/cmparlettpelleriti/CPSC392ParlettPelleriti/master/Data/KMEM3.csv")
d4 = pd.read_csv("https://raw.githubusercontent.com/cmparlettpelleriti/CPSC392ParlettPelleriti/master/Data/KMEM4.csv")
d5 = pd.read_csv("final_3.csv")
fig = px.scatter(d4, x="x", y="y")
fig.show()

In [ ]:
def Agglomerative(n_clusters , linkage_type  ,metric_type , data):    
    Agg = AgglomerativeClustering(n_clusters=n_clusters, linkage = linkage_type , metric= metric_type)
    cluster = Agg.fit(data)
    
    cluster = Agg.fit_predict(data)

    cluster_labels = Agg.labels_

    df1=data.copy()
    df1["Clusters"] = cluster_labels
    df1["Clusters"] = df1["Clusters"].astype('category')
    silhouette = silhouette_score(data, cluster_labels)
    
    mycolor = { 0: '#636EFA',
                1: '#EF553B',
                2: '#00CC96',
                3: '#AB63FA',
                4: '#FFA15A'}
    
    mycolor2 = {'Cluster 0': '#636EFA',
                'Cluster 1': '#EF553B',
                'Cluster 2': '#00CC96',
                'Cluster 3': '#AB63FA',
                'Cluster 4': '#FFA15A'}
    
    fig_scatter = px.scatter(df1, x="x", y="y", color="Clusters", size = cluster_labels+5 , size_max =15 ,
                             color_discrete_map =mycolor ,
                             )
    fig_scatter.update_layout(title="Agglomerative Hierarchical Clustering")


    # Create the pie chart
    #color = [ '#636EFA', '#EF553B','#00CC96', '#AB63FA', '#FFA15A', '#19D3F3', '#FF6692', '#B6E880', '#FF97FF', '#FECB52']

    # Count the number of samples in each cluster
    counts = [sum(df1["Clusters"] == i) for i in range(n_clusters)]
    fig_pie = px.pie(df1, values= counts, names=[f'Cluster {i}' for i in range(n_clusters)],
                 color=[f'Cluster {i}' for i in range(n_clusters)],
                 color_discrete_map = mycolor2,
                 title='Number of Examples in Each Cluster')
    
    
    return fig_scatter , fig_pie

In [ ]:
fig_scatter , fig_pie = Agglomerative(3 , 'single'  ,'euclidean' , d4)

In [ ]:
def Hierarchical(kn,linkage_type  ,metric_type , data):
    Z = hierarchy.linkage(data, method= linkage_type ,metric= metric_type)
    

    #Elbow Method
    last = Z[-10:, 2]
    last_rev = last[::-1]
    idxs = np.arange(1, len(last) + 1)
#     elbow_fig = px.line( x=idxs, y=last_rev, markers=True) #,color_discrete_sequence=['#f4d44d'],
    elbow_fig = go.Figure()
    elbow_fig.add_trace(go.Scatter( x=idxs, y=last_rev))
    acceleration = np.diff(last, 2)  # 2nd derivative of the distances
    acceleration_rev = acceleration[::-1]
    k = acceleration_rev.argmax() + 2  # if idx 0 is the max of this we want 2 clusters

    
    elbow_fig.add_trace(go.Scatter( x=(idxs[:-2] + 1), y=acceleration_rev))
    elbow_fig.update_layout(
        title=f'Elbow Method (Best k : {k})',
        xaxis_title='Number of Clusters',
        yaxis_title= 'distance'
    )

    
    
    max_d=last_rev[kn-1]
    #dendrogram
    dend_fig = plt.figure(figsize=(18, 8))
    plt.title('Hierarchical Clustering Dendrogram',fontsize=20)
    plt.xlabel('sample index or (cluster size)',fontsize=18)
    
    plt.ylabel('distance',fontsize=18)
    hierarchy.dendrogram(
        Z,
        leaf_rotation=90.,
        leaf_font_size=1,
        show_contracted=True, # to get a distribution impression in truncated branches
    )
    plt.axhline(y=max_d, color='r', linestyle='--')
#     dend_fig.show()

    dend_fig.savefig("fig1.png")
    image_url  = "fig1.png"
    img_file = open("fig1.png", "rb")
    my_string = b64encode(img_file.read())
    encoding = my_string.decode()
    img_b64 = "data:image/png;base64," + encoding

    

    return k , elbow_fig ,img_b64 

In [ ]:
k , elbow_fig ,img_b64 = Hierarchical(2,'single'  ,'euclidean' ,d4)#dists


In [ ]:
 elbow_fig

In [ ]:
def dbscan(epsonol, no_samples, df_new):
    scan = DBSCAN(eps =epsonol, min_samples =no_samples).fit(df_new)
    db_scan=df_new.copy()
    db_scan["cluster"]=scan.labels_
    labels = db_scan['cluster'].unique()
    dff_up=df_new.copy()
    dff_up['cluster']=db_scan['cluster'].astype("category")




    n_clusters=len(db_scan['cluster'].unique())
    counts = [sum(db_scan["cluster"] == i) for i in labels]



    # Create scatter plot


    fig = px.scatter(dff_up, x="x", y="y",
              color= 'cluster',
                    size_max=10,size=(db_scan['cluster']+5))
    fig.update_layout(title="DBSCAN Clustering")

    fig_pie = px.pie(names=[f'Cluster {i}' for i in range(n_clusters)],
             values=counts,
             title='Number of Examples in Each Cluster in DBSCAN Clustering',)
    return fig,fig_pie
  

In [ ]:
fig,fig_pie=dbscan(0.55,10,d1)
# fig

In [ ]:
def kmeans_clustering(data, n_clusters):
    x_col = 'x' if 'x' in data.columns else data.columns[0]
    y_col = 'y' if 'y' in data.columns else data.columns[1]

    kmeans = KMeans(n_clusters=n_clusters)

    kmeans.fit(data[[x_col, y_col]])

    labels = kmeans.labels_
    centers = kmeans.cluster_centers_
    df=data.copy()
    df["cluster"]=kmeans.labels_
    df["KMc"]=df["cluster"].astype("category") 
    silhouette_avg = silhouette_score(data[[x_col, y_col]], labels)
    silhouette_values = silhouette_samples(data[[x_col, y_col]], labels)

    scatter_fig = px.scatter(data, x=x_col, y=y_col, color=df["KMc"],size=(df["cluster"]+5),
                             size_max=10)
    scatter_fig.update_layout(title="kmeans Clustering")



    # Create elbow plot
    inertias = []
    for k in range(1, 11):
        kmeans = KMeans(n_clusters=k)
        kmeans.fit(data[[x_col, y_col]])
        inertias.append(kmeans.inertia_)
    elbow_fig = px.line(x=list(range(1, 11)), y=inertias, title='Elbow Plot')
    elbow_fig.update_layout(xaxis_title='Number of Clusters', yaxis_title='Inertia')

    counts = pd.Series(labels).value_counts(normalize=True).reset_index()
    counts.columns = ['cluster', 'percentage']

    percentage_fig = px.pie(counts, values='percentage', names='cluster',
                           title='Number of Examples in Each Cluster',)

    return scatter_fig, percentage_fig, elbow_fig


# Dash

In [ ]:
from dash import html,dcc,Dash
import dash_bootstrap_components as dbc
import plotly.io as pio
from dash.dependencies import Input , Output , State

In [ ]:
#Agglomerative layout

background = '#eadcf4'
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "18rem",
    "padding-top": "12rem ",
    "padding-left": "1rem ",
#     "background-color": background,
}
Tabs_STYLE = {
#     "position": "fixed",
    "right": 0,
    "left": 0,
    "margin-top": "3rem"
#     "background-color": background,
}

# the styles for the main content position it to the right of the sidebar and
CONTENT_STYLE = {
    "background-color": background,
    "padding-right": "8px",
    "padding-top": "8px ",
}



#inputs
Agg_inputs = dbc.Card(dbc.CardBody(html.Div(children=[
    html.Label('Select Dataset: :'),
    html.Div([
        dcc.Dropdown(
            options=['Dataset 1', 'Dataset 2', 'Dataset 3','Dataset 4','Dataset 5'],
            placeholder="Select a Dataset",
            value= 'Dataset 4',
            id='Dataset',
        )
    ],style={'padding-top': '35px','padding': 20}),
    
     html.Br(),
    html.Div([
        html.Label('Clusters :'),
        dcc.Slider(2, 10, 1,
                   value=4,
                   id='k_Slider',
        )
    ],style={'padding-top': 15,'padding-bottom': 15,'padding-right': 5,'padding-left': 5}),
    
    html.Br(),
    
    html.Label('Linkage :'),
    html.Div([
        dcc.RadioItems(
            options=['complete', 'single', 'ward','average'],
            value='complete',
            labelStyle={'display': 'block'},
            id='linkage',
        ),
    ],style={'padding':20,'padding-top': '10px'}),
    
     html.Br(),
    html.Label('Metric :'),
    html.Div([
        dcc.RadioItems(
            options=['euclidean', 'cosine', 'correlation'],
            value='euclidean',
            labelStyle={'display': 'block'},
            id='metric',
        )
    ],style={'padding': 20,'padding-top': '10px'},)

],))       
    , style={"width": "22rem"},
)


# #Sidebar
Agg_sidebar = html.Div(
    [
         dbc.Row([
            dcc.Loading(
                    id="loading-1",
                    type="default",
                    children=html.P(id="loading-output-1")
                ),
            ], align="center",),
        html.Br(),

        html.Div([Agg_inputs])
        
    ],
       style=SIDEBAR_STYLE,
    )




#Body
AggGraphs = html.Div([
    html.Br(),
    html.Br(),
    dbc.Row([
        dbc.Col(dbc.Card(dbc.CardBody(dcc.Graph( id='scatter',figure =fig_scatter))), width=8),
        dbc.Col(dbc.Card(dbc.CardBody(dcc.Graph( id='pie',figure =fig_pie))), width=4)
    ]),

    
    
    html.Br(),
    
     dbc.Row([
        dbc.Col(dbc.Card(dbc.CardBody(html.Img( id='img_hier',
                                               style={'height': '600px','width':'1250px'}))))
     ]),
    html.Br(),
    dbc.Row([
        dbc.Col(dbc.Card(dbc.CardBody(dcc.Graph( id='elbow',figure = elbow_fig))))
    ])

    ] ,style = {"padding-right": "10px",})


# , style=CONTENT_STYLE

In [ ]:
#DBSCAN Clustering

Slider1 = html.Div([
        
        dbc.Label('episonal'),
        dcc.Slider(
            id='Episonal value',
            min=0.1,
            max=1,
            step=0.1,
            value=0.5,
            vertical=True,
            marks={str(i): str(i) for i in range(0, 1)},
            
             ),
    ]),

Slider2 =html.Div([ 
        dbc.Label('min_samplies'),
        dcc.Slider(
            id='Min_samplies',
            min=2,
            max=15,
            step=2,
            value=3,
            vertical=True,
            marks={str(i): str(i) for i in range(0, 15)},
            
        ),
        
        
    ])

DB_inputs =  dbc.Card(dbc.CardBody(html.Div(children=[
    
    
     html.Div(
            [
                dbc.Label("Data_Input"),
                dcc.Dropdown(
                    ['Dataset 1', 'Dataset 2', 'Dataset 3','Dataset 4','Dataset 5'],
                    id="data_set",
                    value="Dataset 4",
                    placeholder="Choose the Data",
                    
                    
                ),
            ],style={'padding': 20} ),
    
    
            dbc.Row([
                dbc.Col(Slider1, width=6),
                dbc.Col(Slider2, width=6)
            ])
    
    
    
        ],))       
    , style={"width": "24rem"},
)

# #Sidebar
DB_sidebar = html.Div(
    [
        
         dbc.Row([
        dcc.Loading(
                id="loading-2",
                type="default",
                children=html.P(id="loading-output-2")
            ),
        ], align="center",),
        html.Br(),
        
        html.Div([DB_inputs])
        
    ],
       style=SIDEBAR_STYLE,
    )


#Body
DBGraphs = html.Div([
    html.Br(),        
    html.Br(),

    dbc.Row([
       dbc.Card(dcc.Graph( id='figure_1'),body=True)]),
    html.Br(),
    dbc.Row([
        dbc.Card(dcc.Graph( id='figure_pie_1'),body=True)])
    ] , style = {"padding-right": "12px",})
    

    



In [ ]:
#kmeans



controls =  dbc.Card(dbc.CardBody(html.Div([
#         html.H3('Select Dataset',style={'fontSize': 13}),
        html.Br(),
         html.Br(),
        dbc.Label("Select Dataset"),
        dcc.Dropdown(
                    ['Dataset 1', 'Dataset 2', 'Dataset 3','Dataset 4','Dataset 5'],
                    id="dataset",
                    value='Dataset 4',
                    placeholder="Choose the Data",
                    
                    
                ),

        html.Br(),
         html.Br(),
        dbc.Label("Select Number of Clusters"),
#         html.H3('Select Number of Clusters',style={'fontSize': 13}),
        dcc.Slider(
            id='cluster-count',
            min=2,
            max=10,
            step=1,
            value=2,
            marks={
                str(i): str(i) for i in range(2, 11)
            }),
         html.Br(),
         html.Br(),
    
]
))       
    , style={"width": "24rem"},
)


# #Sidebar
KM_sidebar = html.Div(
    [
        
        dbc.Row([
        dcc.Loading(
                id="loading-3",
                type="default",
                children=html.P(id="loading-output-3")
            ),
        ], align="center",),
        html.Br(),

        html.Div([controls])
        
    ],
       style=SIDEBAR_STYLE,
    )



KMGraphs = html.Div([
    html.Br(),
    html.Br(),
    dbc.Row([
        dbc.Col(dbc.Card(dbc.CardBody(dcc.Graph(id='scatter-plot'))), width=8),
        dbc.Col(dbc.Card(dbc.CardBody(dcc.Graph(id='cluster-percentage'))), width=4)
    ]),

    html.Br(),
    
    dbc.Row([
        dbc.Col(dbc.Card(dbc.CardBody(dcc.Graph(id='elbow-plot'))))
    ])

    ] , style=CONTENT_STYLE)


In [ ]:
#Navbar
# 'text-align': 'center',
bar="#a72ef9"
navbar = dbc.NavbarSimple(
    brand="Unsupervised Learning",
    brand_style= {'font-size':'30px','fontWeight': 'bold','text-align': 'justify',},
    color=bar,
    dark=True,
    fixed = 'top',
    style = {"height": "6rem",}
)


Agg_layout = dbc.Row([
        dbc.Col(Agg_sidebar, width=2,),
        dbc.Col(AggGraphs, width=10)
    ])




DB_layout = dbc.Row([
        dbc.Col(DB_sidebar, width=2,),
        dbc.Col(DBGraphs, width=10)
    ])

KM_layout = dbc.Row([
        dbc.Col(KM_sidebar, width=2,),
        dbc.Col(KMGraphs, width=10)
    ])




tabs_styles = {
    "right": 0,
    "left": 0,
    "margin-top": "3rem",
    'height': '44px'
}
tab_style = {
    'borderBottom': '1px solid #ffffff',
    'border-radius': '8px',
    'padding': '6px',
    'fontWeight': 'bold'
}

tab_selected_style = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': bar,
    'border-radius': '8px',
    'color': 'white',
    'padding': '6px',
    'fontWeight': 'bold'
}

build_tabs = html.Div(
        id="tabs",
        className="tabs",
        children=[
            dcc.Tabs(
                id="app-tabs",
                children=[
                    dcc.Tab(
                        label="Agglomerative Clustering ",
                        children= [Agg_layout],
                        style=tab_style,
                        selected_style=tab_selected_style
                    ),
                    dcc.Tab(
                        label="DBSCAN Clustering",
                        children= [DB_layout],
                        style=tab_style,
                        selected_style=tab_selected_style
                    ),
                    dcc.Tab(
                        label="KMeans Clustering",
                        children= [KM_layout],
                        style=tab_style,
                        selected_style=tab_selected_style
                    ),
                ],style=tabs_styles
            )
        ] 
    )

body_layout = html.Div([
     dbc.Row([
        html.Br(),
        build_tabs,
    ]),

])





In [ ]:
app = Dash(external_stylesheets=[dbc.themes.BOOTSTRAP,'https://codepen.io/chriddyp/pen/bWLwgP.css'])
app.layout = html.Div([
    navbar,
    html.Br(),
    dbc.Row([
        dbc.Col(body_layout)
    ])
], style = {"background-color": background,})
  


    
# callback tab1 Agglomerative
@app.callback(
    Output('scatter', 'figure'),
    Output('pie', 'figure'),
    Output('img_hier', 'src'),
    Output('elbow', 'figure'),
    Output("loading-output-1", "children"),
    Input('Dataset', 'value'),
    Input('k_Slider', 'value'),
    Input('linkage', 'value'),
    Input('metric', 'value')
)
def UpdateALL(Dataset ,k,linkage,metric):
    if Dataset == 'Dataset 1':
        data = d1
    elif Dataset == 'Dataset 2':
        data = d2
    elif Dataset == 'Dataset 3':
        data = d3
    elif Dataset == 'Dataset 4':
        data = d4
    else :
        data = d5

    fig_scatter , fig_pie = Agglomerative(k , linkage  ,metric , data)
    k , elbow_fig ,image_url = Hierarchical(k,linkage ,metric ,data)


    return fig_scatter, fig_pie, image_url, elbow_fig," "


# callback tab2  
@app.callback(
    Output('figure_1', 'figure'),
    Output('figure_pie_1', 'figure'),
    Output("loading-output-2", "children"),
    Input('app-tabs', 'value'),
    Input('Episonal value', 'value'),
    Input('Min_samplies', 'value'),
    Input('data_set', 'value')
#     prevent_initial_call=True
)

def UpdateALL(tab,eps_val, samples_val,dt):
    if dt == 'Dataset 1':
        data = d1
    elif dt == 'Dataset 2':
        data = d2
    elif dt == 'Dataset 3':
        data = d3
    elif dt == 'Dataset 4':
        data = d4
    else :
        data = d5
    
    fig, fig_pie = dbscan(eps_val, samples_val, data) 
    return fig, fig_pie,""

# callback tab3  
@app.callback(
     Output('scatter-plot', 'figure'),
     Output('cluster-percentage', 'figure'),
     Output('elbow-plot', 'figure'),
     Output("loading-output-3", "children"),
     Input('dataset', 'value'),
     Input('cluster-count', 'value')
)
def update_graphs(dataset, n_clusters):
    if dataset == 'Dataset 1':
        data = d1
    elif dataset == 'Dataset 2':
        data = d2
    elif dataset == 'Dataset 3':
        data = d3
    elif dataset == 'Dataset 4':
        data = d4
    else :
        data = d5
        
    scatter_fig,percentage_fig, elbow_fig = kmeans_clustering(data, n_clusters)
    
    return scatter_fig, percentage_fig, elbow_fig,""
    
app.run_server()

## Done :)